# Practice with NYTimes API
CS315: Data Science for the Web  
Professor Eni Mustafaraj  
[Day 10 Slides 22-28](https://docs.google.com/presentation/d/15fuBhqPNv8GgeqNlNydKAlJW5ecAx92h7lQ_KyTxbKI/edit#slide=id.g2bd1c28673c_0_127)  
Edith Po  

In [2]:
import requests

In [13]:
key = '1DFmIMxxqdYl8wJBPqAFxtHkimk86Qtn'

In [15]:
year = 2024
month = 2

url = f'https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={key}'
print(url)

https://api.nytimes.com/svc/archive/v1/2024/2.json?api-key=1DFmIMxxqdYl8wJBPqAFxtHkimk86Qtn


In [16]:
data = requests.get(url)
data.status_code

200

In [18]:
articles = data.json()
len(articles)

2

In [23]:
articles.keys()

dict_keys(['copyright', 'response'])

In [26]:
len(articles['response']['docs'])

3791

In [31]:
articles['response']['docs'][0].keys()

dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section', 'print_page', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])

In [33]:
article0 = articles['response']['docs'][0]
print(article0['abstract'])
print(article0['web_url'])
print(article0['snippet'])
print(article0['lead_paragraph'])
print(article0['keywords'])

Periods of backlash take shape after surges of Black progress. We have entered another such period.
https://www.nytimes.com/2024/01/31/opinion/racist-backlash-history.html
Periods of backlash take shape after surges of Black progress. We have entered another such period.
I am fascinated, and alarmed, by the swiftness with which periods of backlash take shape after surges of Black progress, and I believe that we have entered another such period.
[{'name': 'subject', 'value': 'Hate Crimes', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Black People', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Blacks', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'Discrimination', 'rank': 4, 'major': 'N'}, {'name': 'subject', 'value': 'Civil Rights Movement (1954-68)', 'rank': 5, 'major': 'N'}, {'name': 'subject', 'value': 'Reconstruction Era', 'rank': 6, 'major': 'N'}, {'name': 'subject', 'value': 'Segregation and Desegregation', 'rank': 7, 'major': 'N'}, {'name': 'person